In [1]:
# Essential libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek 
from sklearn.preprocessing import OneHotEncoder
from collections import Counter

import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Bidirectional, LSTM
from keras.optimizers import SGD
from keras import regularizers
from keras.callbacks import LearningRateScheduler

In [2]:
# Importing training and test data
raw_data = pd.read_csv('train.csv', header=None, index_col=0)
testing = pd.read_csv('test.csv',header=None, index_col=0 )

In [3]:
# Splitting features and labels
dataset = raw_data.values
X = dataset[:, 0:960].astype(float)
Y = dataset[:, 960]

In [4]:
# Oversampling minority classes
sm = SMOTE(k_neighbors=11, random_state=42)
sm1 = SMOTETomek(smote=sm,random_state=42)

X, Y = sm1.fit_resample(X,Y)

In [5]:
scaler = RobustScaler()
scaler = scaler.fit(X)
X = scaler.transform(X)

In [6]:
Y = Y.reshape(len(Y), 1)

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
matrix_Y = enc.fit_transform(Y)

In [7]:
# Spliting training and validation data
x_train, x_test, y_train, y_test = train_test_split(X, matrix_Y, test_size=0.3, random_state=42, stratify=matrix_Y)

In [8]:
# Reshaping to use as input in CNN
x_train = x_train.reshape(len(x_train), 16, 60, 1)
x_test = x_test.reshape(len(x_test), 16, 60, 1)

In [9]:
keras.backend.clear_session()

def lr_schedule(epoch):
    '''
    Returning learning rate according to the epoch number
    '''
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 125:
        lrate = 0.0003       
    return lrate


def ReshapeLayer(x): 
    '''
    Reshaping to 2D array to use as input
    in Bi-LSTM layer
    '''
    shape = x.shape  
    reshape = keras.layers.Reshape((shape[1], shape[2]*shape[3]))(x)
    
    return reshape

weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', 
                 bias_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(weight_decay), 
                 input_shape=(16, 60, 1)
                )
         )
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', 
                 bias_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(weight_decay)
                )
         )
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', 
                 bias_initializer='glorot_uniform', 
                 kernel_regularizer=regularizers.l2(weight_decay)
                )
         )
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', 
                 bias_initializer='glorot_uniform', 
                 kernel_regularizer=regularizers.l2(weight_decay)
                )
         )
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', 
                 bias_initializer='glorot_uniform', 
                 kernel_regularizer=regularizers.l2(weight_decay)
                )
         )
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', 
                 bias_initializer='glorot_uniform', 
                 kernel_regularizer=regularizers.l2(weight_decay)
                )
         )
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(keras.layers.Lambda(ReshapeLayer))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(96)))
model.add(Dropout(0.2))
 
model.add(Flatten())
model.add(Dense(49, activation='softmax'))
 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 60, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 16, 60, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 60, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 60, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 16, 60, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 60, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 30, 32)         0

In [10]:
#training
batch_size = 64
 
# Defining SGD optimizer
opt = SGD(lr=0.001, momentum=0.9)

earlystopping = keras.callbacks.EarlyStopping(monitor="val_loss", 
                                              mode="min", patience=25, 
                                              restore_best_weights=True
                                             )

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Fitting the model
history = model.fit(x_train, y_train, 
                    batch_size=batch_size, 
                    epochs=200, 
                    verbose=1,
                    validation_data=(x_test,y_test),
                    callbacks=[LearningRateScheduler(lr_schedule), earlystopping]
                   )

Epoch 1/200
316/316 [==============================] - 88s 279ms/step - loss: 3.8985 - accuracy: 0.0328 - val_loss: 3.8145 - val_accuracy: 0.0610
Epoch 2/200
316/316 [==============================] - 82s 259ms/step - loss: 3.7971 - accuracy: 0.0545 - val_loss: 3.6872 - val_accuracy: 0.0654
Epoch 3/200
316/316 [==============================] - 82s 259ms/step - loss: 3.6826 - accuracy: 0.0627 - val_loss: 3.5747 - val_accuracy: 0.0810
Epoch 4/200
316/316 [==============================] - 85s 269ms/step - loss: 3.5780 - accuracy: 0.0785 - val_loss: 3.4772 - val_accuracy: 0.0975
Epoch 5/200
316/316 [==============================] - 80s 254ms/step - loss: 3.4932 - accuracy: 0.0945 - val_loss: 3.3915 - val_accuracy: 0.1415
Epoch 6/200
316/316 [==============================] - 80s 253ms/step - loss: 3.4149 - accuracy: 0.1173 - val_loss: 3.3032 - val_accuracy: 0.1659
Epoch 7/200
316/316 [==============================] - 80s 255ms/step - loss: 3.3395 - accuracy: 0.1359 - val_loss: 3.2224 -

Epoch 57/200
316/316 [==============================] - 82s 260ms/step - loss: 0.8685 - accuracy: 0.7666 - val_loss: 0.7479 - val_accuracy: 0.8031
Epoch 58/200
316/316 [==============================] - 83s 264ms/step - loss: 0.8551 - accuracy: 0.7698 - val_loss: 0.7246 - val_accuracy: 0.8102
Epoch 59/200
316/316 [==============================] - 82s 260ms/step - loss: 0.8318 - accuracy: 0.7746 - val_loss: 0.7246 - val_accuracy: 0.8084
Epoch 60/200
316/316 [==============================] - 83s 262ms/step - loss: 0.8181 - accuracy: 0.7775 - val_loss: 0.7125 - val_accuracy: 0.8122
Epoch 61/200
316/316 [==============================] - 83s 262ms/step - loss: 0.7968 - accuracy: 0.7853 - val_loss: 0.7055 - val_accuracy: 0.8132
Epoch 62/200
316/316 [==============================] - 81s 255ms/step - loss: 0.7742 - accuracy: 0.7925 - val_loss: 0.6769 - val_accuracy: 0.8229
Epoch 63/200
316/316 [==============================] - 83s 262ms/step - loss: 0.7666 - accuracy: 0.7900 - val_loss: 0

316/316 [==============================] - 81s 257ms/step - loss: 0.4063 - accuracy: 0.8949 - val_loss: 0.5070 - val_accuracy: 0.8762
Epoch 113/200
316/316 [==============================] - 81s 257ms/step - loss: 0.4038 - accuracy: 0.8935 - val_loss: 0.5045 - val_accuracy: 0.8754
Epoch 114/200
316/316 [==============================] - 81s 257ms/step - loss: 0.3999 - accuracy: 0.8935 - val_loss: 0.4937 - val_accuracy: 0.8807
Epoch 115/200
316/316 [==============================] - 81s 257ms/step - loss: 0.4050 - accuracy: 0.8911 - val_loss: 0.5024 - val_accuracy: 0.8791
Epoch 116/200
316/316 [==============================] - 81s 257ms/step - loss: 0.3904 - accuracy: 0.8967 - val_loss: 0.4973 - val_accuracy: 0.8807
Epoch 117/200
316/316 [==============================] - 81s 257ms/step - loss: 0.3919 - accuracy: 0.8966 - val_loss: 0.5016 - val_accuracy: 0.8787
Epoch 118/200
316/316 [==============================] - 82s 259ms/step - loss: 0.3891 - accuracy: 0.8963 - val_loss: 0.4946 -

316/316 [==============================] - 84s 265ms/step - loss: 0.2680 - accuracy: 0.9316 - val_loss: 0.4678 - val_accuracy: 0.8927
Epoch 168/200
316/316 [==============================] - 85s 270ms/step - loss: 0.2690 - accuracy: 0.9315 - val_loss: 0.4833 - val_accuracy: 0.8920
Epoch 169/200
316/316 [==============================] - 84s 267ms/step - loss: 0.2637 - accuracy: 0.9339 - val_loss: 0.4825 - val_accuracy: 0.8919
Epoch 170/200
316/316 [==============================] - 86s 274ms/step - loss: 0.2669 - accuracy: 0.9307 - val_loss: 0.4778 - val_accuracy: 0.8927
Epoch 171/200
316/316 [==============================] - 86s 272ms/step - loss: 0.2598 - accuracy: 0.9340 - val_loss: 0.4732 - val_accuracy: 0.8927
Epoch 172/200
316/316 [==============================] - 84s 264ms/step - loss: 0.2570 - accuracy: 0.9352 - val_loss: 0.4658 - val_accuracy: 0.8956
Epoch 173/200
316/316 [==============================] - 85s 270ms/step - loss: 0.2606 - accuracy: 0.9341 - val_loss: 0.4768 -

In [16]:
# Reshaping the test data
sample_data = scaler.transform(testing)
sample_data = sample_data.reshape(len(sample_data), 16, 60, 1)

# Predicting the test data
y_pred = model.predict(sample_data)

# Converting one hot encoded values back to categorical labels
y_pred = enc.inverse_transform(y_pred)

In [17]:
# Reading the sample csv file to replicate a file like that for test prediction data
sample = pd.read_csv('sample.csv')
submission = pd.DataFrame(columns=sample.columns)
submission['Id']=sample['Id']
submission['Category']=pd.DataFrame(y_pred)

In [18]:
submission.to_csv("final_model.csv", index=False)